In [123]:
import numpy as np
import tensorflow as tf

In [124]:
def binary_encoding(i, num_digits):
#     print("NP.ARRAY", np.array)
    return np.array([i >> d % 1 for d in range(num_digits)])

def fizz_buzz_encoding(i):
    if 1 % 15 == 0:
        return np.array([0,0,0,1])
    elif i % 5 == 0: return np.array([0,0,1,0])
    elif i % 3 == 0: return np.array([0,1,0,0])
    else:
        return np.array([1,0,0,0])

In [125]:
NUM_DIGITS = 10
training_X = np.array([binary_encoding(i, NUM_DIGITS) for i in range(101, 2** NUM_DIGITS)])
print(len(training_X))
print(training_X[0])
training_Y = np.array([fizz_buzz_encoding(i) for i in range(101, 2 ** NUM_DIGITS)])
print(len(training_Y))
print(training_Y[0])

923
[101 101 101 101 101 101 101 101 101 101]
923
[1 0 0 0]


In [126]:
HIDDEN_LAYER = 100

X = tf.placeholder("float", [None, NUM_DIGITS])
Y = tf.placeholder("float", [None, 4])

In [127]:
def initialize_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

weights_hidden_layer = initialize_weights([NUM_DIGITS, HIDDEN_LAYER])
weights_output_layer = initialize_weights([HIDDEN_LAYER, 4])

In [128]:
# Defining the Model.
def model(X, weights_hidden_layer, weights_output_layer):
    h = tf.nn.relu(tf.matmul(X, weights_hidden_layer))
    return tf.matmul(h, weights_output_layer)

In [129]:
py_x = model(X, weights_hidden_layer, weights_output_layer)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

In [130]:
predict_op = tf.argmax(py_x, 1)

In [131]:
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [ ]:
batch_size = 128

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for epoch in range(10000):
        p = np.random.permutation(range(len(training_X)))
        training_X, training_Y = training_X[p], training_Y[p]
        
        for start in range(0, len(training_X), batch_size):
            end = start + batch_size
            sess.run(train_op, feed_dict={X: training_X[start:end], Y: training_Y[start:end]})
        
        print(epoch, np.mean(np.argmax(training_Y, axis=1) == sess.run(predict_op, feed_dict={X: training_X, Y: training_Y})))
    
    numbers = np.arange(1,101)
    test_X = np.transpose(binary_encoding(numbers, NUM_DIGITS))
    test_Y = sess.run(predict_op, feed_dict={X: test_X})
    output = np.vectorize(fizz_buzz)(numbers, test_Y)
    
    print(output)